<a href="https://colab.research.google.com/github/41371131h-chi/114-1-/blob/main/HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
!pip install folium

In [56]:
import folium
lat = 25.027472815739536
lng = 121.52993850781064
m = folium.Map(location=(lat, lng), zoom_start=15)

In [57]:
# ==============================================================================
# 0. 環境設置與函式庫導入
# ==============================================================================
# --- 運行環境設定（請在 Colab Cell 中執行）---
!pip -q install gspread gspread_dataframe google-auth google-auth-oauthlib google-auth-httplib2 \
              gradio pandas beautifulsoup4 google-generativeai python-dateutil scikit-learn

In [58]:
import os, time, uuid, re, json, datetime
from datetime import datetime as dt, timedelta
from dateutil.tz import gettz
import pandas as pd
import gradio as gr
import requests
from bs4 import BeautifulSoup
import google.generativeai as genai

# Google Auth & Sheets
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from google.auth import default

In [59]:
# 認證與 Gemini 配置（請在 Colab Cell 中執行）
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

from google.colab import userdata
# 從 Colab Secrets 中獲取 API 金鑰
try:
    api_key = userdata.get('HW3')
    # 確保金鑰存在，否則 genai.configure 會報錯
    if not api_key:
        raise ValueError("Colab Secret 'HW3' is empty or not found.")

    genai.configure(api_key=api_key)
    print("✅ Gemini API Key 配置成功。")
except Exception as e:
    print(f"⚠️ Gemini API Key 配置失敗，請檢查 Colab Secrets 中的 'HW3'：{e}")

✅ Gemini API Key 配置成功。


In [60]:
# ==============================================================================
# 1. 全域變數與 Sheet/DataFrame 設置
# ==============================================================================

# 請確保這個 Sheet URL 存在且您有編輯權限
SHEET_URL = "https://docs.google.com/spreadsheets/d/1nuhUMYejx4I0uq-2Fro6Bjlpd0H1VW0ecVyKXM1AePA/edit?usp=sharing"
WORKSHEET_NAME = "food_choice"
TIMEZONE = "Asia/Taipei"

In [61]:
# 從 gsheets 的 All-whiteboard-device 載入 sheets
gsheets = gc.open_by_url(SHEET_URL)
sheets = gsheets.worksheet(WORKSHEET_NAME).get_all_values()
# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df_restaurants = pd.DataFrame(sheets[1:], columns=sheets[0])
# 取得最前面的5筆資料
df_restaurants.head()

,店家名稱,類別/招牌品項,參考地點,備註/特色,lat,lng
0,難吃,便當菜/快餐,師大,牛肉麵,25.02207668,121.5283606
1,溏老鴨平價小火鍋,小火鍋,新生南路,平實的餐廳，專售海鮮、肉類和蔬菜火鍋料理，並供應家常甜點。,25.01981157,121.5334397
2,大埔鐵板燒,日式料理,公館,沒有免費無限加菜了,25.01239099,121.5355401
3,SUKIYA,日式料理,公館,快速出餐,25.01767512,121.5318739
4,鍋 in,小火鍋,公館,人一直都很多的地方,25.012641,121.535253


In [62]:
for i in range(0, len(df_restaurants)):
  lat = df_restaurants['lat'][i]
  lng = df_restaurants['lng'][i]
  tooltip_str = df_restaurants['店家名稱'][i]
  popup_str = df_restaurants['類別/招牌品項'][i]

  folium.Marker(
      location = [lat, lng],
      tooltip = tooltip_str,
      popup = popup_str,
      icon=folium.Icon(icon="cloud"),
  ).add_to(m)


In [63]:
m